# Data Engineering - Assignment 04
## US Top Music Schools Impacted by Severe Winter Weather (January 2026)

**Author:** Greg Sullivan

**Date:** February 2026

**Course:** DATA 5035 - Data Engineering

**Professor:** Paul Boal

---

### Objective
Build a data pipeline that combines music conservatory data (from HTML scraping and APIs) with weather data (from API) to estimate student-days impacted by severe winter weather in January 2026.

### My Selected Grouping
**Top 10 Music Schools in the United States**

Source: [www.thebestschools.org](http://www.thebestschools.org/) - "The Best Music Schools" ranking

### Severe Weather Definition
**A day is considered "severe winter weather" if it meets ANY of:**
- Minimum temperature below 20°F (-6.7°C)
- Maximum temperature below 32°F (0°C) for entire day
- Precipitation > 0.5 inches while temperature below 32°F (ice/snow)
(That's my Midwestern view.  When I worked in Florida I would've had a completely different view!)

### Data Collection Strategy
**Where options existed, I attempted to collect data in this order:**
1. **Web Scraping** - Extract directly from school websites
2. **API** - College Scorecard API for standalone music schools
3. **Fallback** - Verified data from Common Data Sets (manually entered as a last resort)

### Data Engineering Challenges

**I encountered real-world data collection challenges:**
- **Network Restrictions:** University policy on cloud platforms had restrictions
- **API Rate Limits:** College Scorecard API had request limitations (frustrating!)
- **Inconsistent HTML:** Each school website has unique structure - nothing was consistent
- **Data in PDFs:** Some schools publish enrollment only in PDF documents or other image formats
- **403 Errors:** Some sites block automated requests, even when permission was granted

**Why this matters:** A simple Google search shows enrollment data instantly, but programmatic access required navigating rate limits, firewalls, and inconsistent data formats. 

I now understand why I've heard data engineers say "80% of the work is getting the data."  This was certainly the case for me on this assignment!

---
## Step 1: Import Libraries

In [ ]:
import requests                             # for my HTTP requests to APIs
import pandas as pd
from bs4 import BeautifulSoup               # for web scraping (will try first)
import json
from datetime import datetime, timedelta
import time
import re                                   # regex for pattern matching in what we grab

# At the end I added a few visualizations
import matplotlib.pyplot as plt             # for visualizations

print("Libraries imported successfully")

---
## Step 2: Load School URLs from File

# I created a reference file for the top 10 music schools.  For each school (row in the file) I gathered from general web searches the information I might need later for our analysis.
# These are all needed in case other attempts to find the data fail.  Regardless of the state of technical matters at the time of execution, I want to assure the analysis still completes.
# For each school, I captured the following information:
#   - url
#   - school_name
#   - city
#   - state
#   - latitude
#   - longitude
#   - enrollment (for just the music school, if part of a larger university)
#   - enrollment_source
#   - enrollment_year

These will only be used if our attempts to scrape websites or query APIs fail...

In [ ]:
# Load reference data (contains URLs and all fallback data)
reference_file = 'Music Schools Reference Data.csv'
reference_df = pd.read_csv(reference_file)

print(f"Loaded {len(reference_df)} schools from reference file:\n")
for i, row in reference_df.iterrows():
    print(f"  {i+1}. {row['url']} {row['school_name']} ({row['city']}, {row['state']})")

# Extract URLs for scraping attempts
school_urls = reference_df['url'].tolist()

---
## Step 3: Scrape School Names

In [ ]:
# Function for scraping school name from a given URL
#   Use reference data if unable to scrape school name
#   Show results after attempts
def scrape_school_name(url):
    """
    Scrape school name from website using multiple strategies
    
    Parameters:
    -----------
    url : str
        School website URL
    
    Returns:
    --------
    tuple (str or None, str)
        School name and source/error message
    """
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        response = requests.get(url, timeout=10, headers=headers)
        if response.status_code != 200:
            return None, f'HTTP {response.status_code}'
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Strategy 1: Meta tags
        for attr, value in [('property', 'og:site_name'), ('property', 'og:title')]:
            meta = soup.find('meta', {attr: value})
            if meta and meta.get('content'):
                name = meta.get('content').strip()
                name = re.sub(r'\s*[|•\-–]\s*(Home|About|Welcome).*$', '', name, flags=re.IGNORECASE)
                if name and len(name) > 3 and name.lower() not in ['home', 'about']:
                    return name, 'meta tag'
        
        # Strategy 2: Title tag
        title = soup.find('title')
        if title:
            name = title.get_text(strip=True)
            name = re.sub(r'\s*[|•\-–]\s*(Home|About|Welcome|Official).*$', '', name, flags=re.IGNORECASE)
            if name and len(name) > 3 and name.lower() not in ['home', 'about']:
                return name, 'title tag'
        
        return None, 'no name found'
        
    except Exception as e:
        return None, f'Error: {e}'


# Fallback school names from reference file (no hardcoding!)
fallback_names = dict(zip(reference_df['url'], reference_df['school_name']))


# Scrape school names
print("Scraping school names...\n")
schools_data = []

for url in school_urls:
    print(f"Processing {url}...")
    
    name, source = scrape_school_name(url)

    # Scrape first, otherwise use fallback from reference file
    if name:
        print(f"  ✓ SCRAPED: {name} ({source})")
        name_source = f'scraped ({source})'
    else:
        name = fallback_names.get(url, 'Unknown')
        print(f"  → FALLBACK: {name} ({source})")
        name_source = f'fallback ({source})'
    
    schools_data.append({
        'url': url,
        'name': name,
        'name_source': name_source
    })
    
    time.sleep(1)

schools_df = pd.DataFrame(schools_data)

scraped_count = schools_df['name_source'].str.contains('scraped', case=False).sum()
print(f"\nSchool Names: {scraped_count}/10 scraped, {10-scraped_count}/10 fallback")

---
## Steps 4: Collect Location and Enrollment Data

In [ ]:
# =============================================================================
# Step 4: Collect Location and Enrollment Data
# =============================================================================
# Priority for Enrollment: 1) Web Scraping → 2) College Scorecard API → 3) Reference File
# Priority for Location: 1) College Scorecard API → 2) Reference File
#
# Note: Scraping uses generic patterns that work across most school websites.
# No site-specific configuration - if generic scraping fails, we fall back to API or reference.

#  This proved incredibly challenging.  I eventually settled on this approach:
#    1) Try web scraping with Beautiful Soup (lots of experimentation and tuning of the code);
#    2) If web scraping failed, attempt to gather via the College Scorecard API; and
#    3) If both of those failed, do a manual Google search and hardcode the enrollment data.

#  One challenge I ran into was in the case a music school was only a part of a larger university.
#  My intent is to study for this assignment, MUSIC STUDENTS impacted by winter weather, since
#    they have speciall challenges carrying those instruments around, keeping their hands warm, etc.


#  I used some trial-and-error on where best to find the enrollment data as
#    it was NOT in one consistent location on each university's website


def scrape_enrollment_from_website(url):
    """
    Scrape enrollment using generic patterns (no site-specific config)
    Tries common pages and common enrollment text patterns
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    # Generic pages where enrollment data commonly appears
    common_paths = ['', '/about', '/facts', '/admissions', '/about/facts-and-statistics']
    
    # Generic patterns that match enrollment text across many sites
    generic_patterns = [
        r'student\s+body\s+of\s+(\d+)',
        r'enrollment[:\s]+([0-9,]+)',
        r'([0-9,]+)\s+students\s+(?:are\s+)?enrolled',
        r'(\d+)\s+(?:total\s+)?students',
        r'approximately\s+([0-9,]+)\s+students',
        r'around\s+(\d+)\s+students',
        r'community\s+of\s+([0-9,]+)\s+students',
        r'([0-9,]+)\s+collegiate\s+students',
    ]
    
    # Extract base URL
    from urllib.parse import urlparse
    parsed = urlparse(url)
    base_url = f"{parsed.scheme}://{parsed.netloc}"
    
    for path in common_paths:
        try:
            page_url = base_url + path
            response = requests.get(page_url, timeout=10, headers=headers)
            if response.status_code != 200:
                continue
            
            soup = BeautifulSoup(response.content, 'html.parser')
            text = soup.get_text()
            
            for pattern in generic_patterns:
                matches = re.findall(pattern, text, re.IGNORECASE)
                for match in matches:
                    try:
                        num = int(match.replace(',', '').replace('+', ''))
                        # Reasonable enrollment range for music schools
                        if 50 < num < 10000:
                            return num, f'scraped ({page_url})'
                    except:
                        continue
        except:
            continue
    
    return None, 'Scraping failed (no pattern matched)'


def fetch_from_college_scorecard(school_name):
    """
    Fetch location AND enrollment from College Scorecard API
    """
    # Build search term by removing common music school suffixes
    search_term = school_name
    for suffix in [' School of Music', ' College of Music', ' Conservatory of Music', ' Institute of Music']:
        search_term = search_term.replace(suffix, '')
    search_term = search_term.strip()
    
    api_url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json'
    params = {
        'school.name': search_term,
        'fields': 'school.name,school.city,school.state,location.lat,location.lon,latest.student.size',
        'api_key': 'DEMO_KEY'
    }
    
    try:
        response = requests.get(api_url, params=params, timeout=10)
        if response.status_code == 429:
            return None, 'API rate limited'
        if response.status_code != 200:
            return None, f'HTTP {response.status_code}'
        
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            return None, 'No API results'
        
        r = results[0]
        return {
            'api_name': r.get('school.name', ''),
            'city': r.get('school.city'),
            'state': r.get('school.state'),
            'latitude': r.get('location.lat'),
            'longitude': r.get('location.lon'),
            'enrollment': r.get('latest.student.size')
        }, 'API'
        
    except Exception as e:
        return None, f'API error: {e}'


# Build lookup from reference file (by URL)
ref_by_url = reference_df.set_index('url').to_dict('index')

print("="*70)
print("LOCATION AND ENROLLMENT DATA COLLECTION")
print("="*70)
print("\nEnrollment Priority: 1) Scrape → 2) API → 3) Reference File")
print("Location Priority: 1) API → 2) Reference File\n")

results = []

for idx, row in schools_df.iterrows():
    school_name = row['name']
    url = row['url']
    ref_data = ref_by_url.get(url, {})
    
    # Check if API returns standalone school data (from reference file)
    is_standalone = ref_data.get('is_standalone', True)
    
    print(f"[{idx+1}/{len(schools_df)}] {school_name}")
    
    # Initialize
    city = state = latitude = longitude = enrollment = None
    enrollment_source = location_source = None
    
    # --- ENROLLMENT: Priority 1 - Scrape ---
    enrollment, scrape_result = scrape_enrollment_from_website(url)
    if enrollment:
        enrollment_source = scrape_result
        print(f"      ✓ Enrollment SCRAPED: {enrollment:,}")
    else:
        print(f"      Enrollment scrape: {scrape_result}")
    
    # --- API for location and enrollment (if standalone school) ---
    if is_standalone:
        api_data, api_result = fetch_from_college_scorecard(school_name)
        
        if api_data:
            # Location from API
            city = api_data.get('city')
            state = api_data.get('state')
            latitude = api_data.get('latitude')
            longitude = api_data.get('longitude')
            if city and state:
                location_source = f"API ({api_data.get('api_name', '')})"
                print(f"      ✓ Location from API: {city}, {state}")
            
            # Enrollment from API (if not already scraped)
            if not enrollment and api_data.get('enrollment'):
                enrollment = api_data.get('enrollment')
                enrollment_source = f"API ({api_data.get('api_name', '')})"
                print(f"      ✓ Enrollment from API: {enrollment:,}")
        else:
            print(f"      API: {api_result}")
    else:
        print(f"      API: Skipped (returns university-level data, not music school)")
    
    # --- FALLBACK to reference file ---
    if not city or not state:
        city = ref_data.get('city')
        state = ref_data.get('state')
        latitude = ref_data.get('latitude')
        longitude = ref_data.get('longitude')
        location_source = 'reference file'
        print(f"      → Location FALLBACK: {city}, {state}")
    
    if not enrollment:
        enrollment = ref_data.get('enrollment')
        enrollment_source = 'reference file'
        print(f"      → Enrollment FALLBACK: {enrollment:,}")
    
    results.append({
        'url': url,
        'name': school_name,
        'name_source': row['name_source'],
        'city': city,
        'state': state,
        'latitude': latitude,
        'longitude': longitude,
        'location_source': location_source,
        'enrollment': enrollment,
        'enrollment_source': enrollment_source
    })
    
    time.sleep(1)

# Update schools_df with complete data
schools_df = pd.DataFrame(results)

# Summary
print("\n" + "="*70)
print("RESULTS SUMMARY")
print("="*70)

loc_api = schools_df['location_source'].str.contains('API', case=False, na=False).sum()
loc_ref = schools_df['location_source'].str.contains('reference', case=False, na=False).sum()
print(f"\nLocation:   {loc_api}/10 from API, {loc_ref}/10 from reference file")

enr_scraped = schools_df['enrollment_source'].str.contains('scraped', case=False, na=False).sum()
enr_api = schools_df['enrollment_source'].str.contains('API', case=False, na=False).sum()
enr_ref = schools_df['enrollment_source'].str.contains('reference', case=False, na=False).sum()
print(f"Enrollment: {enr_scraped}/10 scraped, {enr_api}/10 from API, {enr_ref}/10 from reference file")

print("\n")
schools_df[['name', 'city', 'state', 'enrollment', 'enrollment_source', 'location_source']]

---
## Step 5: Fetch Weather Data

In [ ]:
# Collect the weather data, based on location, from the Open-Meteo API

def fetch_weather_data(latitude, longitude, school_name):
    """
    Fetch weather from Open-Meteo API for January 2026
    """
    url = 'https://archive-api.open-meteo.com/v1/archive'
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'start_date': '2026-01-01',
        'end_date': '2026-01-31',
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'temperature_unit': 'fahrenheit',
        'precipitation_unit': 'inch',
        'timezone': 'America/New_York'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        daily_data = data.get('daily', {})
        
        # List comprehension for efficiency
        weather_records = [
            {
                'school_name': school_name,
                'date': date,
                'temp_max_f': temp_max,
                'temp_min_f': temp_min,
                'precipitation_inches': precip
            }
            for date, temp_max, temp_min, precip in zip(
                daily_data.get('time', []),
                daily_data.get('temperature_2m_max', []),
                daily_data.get('temperature_2m_min', []),
                daily_data.get('precipitation_sum', [])
            )
        ]
        
        print(f"  {school_name}")
        return pd.DataFrame(weather_records)
        
    except Exception as e:
        print(f"  {school_name}: Error - {e}")
        return pd.DataFrame()


print("Fetching weather data...\n")
all_weather_data = []

for idx, row in schools_df.iterrows():
    weather_df = fetch_weather_data(
        row['latitude'],
        row['longitude'],
        row['name']
    )
    
    if not weather_df.empty:
        all_weather_data.append(weather_df)
    
    time.sleep(0.5)

weather_data_df = pd.concat(all_weather_data, ignore_index=True)

print(f"\nTotal weather records: {len(weather_data_df)}")

---
## Step 6: Identify Severe Weather Days

In [ ]:
# Use vectorized operations, set severe weather from my documented standards above
weather_data_df['extreme_cold'] = weather_data_df['temp_min_f'] < 20
weather_data_df['full_freeze'] = weather_data_df['temp_max_f'] < 32
weather_data_df['winter_precip'] = (
    (weather_data_df['precipitation_inches'] > 0.5) & 
    (weather_data_df['temp_max_f'] < 32)
)

weather_data_df['is_severe_weather'] = (
    weather_data_df['extreme_cold'] | 
    weather_data_df['full_freeze'] | 
    weather_data_df['winter_precip']
)

severe_weather_df = weather_data_df[weather_data_df['is_severe_weather']].copy()

print(f"Total severe weather days: {len(severe_weather_df)}\n")
print("By school:")
print(severe_weather_df.groupby('school_name')['date'].count().sort_values(ascending=False))

---
## Step 7: Aggregate Severe Weather

In [ ]:
#  Aggregate and count severe weather days
severe_weather_agg = (
    severe_weather_df
    .groupby('school_name')
    .agg({
        'date': lambda dates: ', '.join(sorted(dates)),
        'is_severe_weather': 'count'
    })
    .rename(columns={
        'date': 'severe_weather_dates',
        'is_severe_weather': 'severe_weather_days_count'
    })
    .reset_index()
)

severe_weather_agg

---
## Step 8: Create Final Output

In [ ]:
# schools_df already has enrollment, just rename and merge weather
final_table = schools_df.rename(columns={'name': 'school_name'}).copy()

# Merge with severe weather aggregation
final_table = final_table.merge(severe_weather_agg, on='school_name', how='left')

# Fill NaN for schools with no severe weather
final_table['severe_weather_days_count'] = final_table['severe_weather_days_count'].fillna(0).astype(int)
final_table['severe_weather_dates'] = final_table['severe_weather_dates'].fillna('None')

# Calculate student-days impacted
final_table['student_days_impacted'] = final_table['enrollment'] * final_table['severe_weather_days_count']

# Sort by count of impacted days
final_table = final_table.sort_values('student_days_impacted', ascending=False).reset_index(drop=True)

print("="*80)
print("FINAL RESULTS: Music Schools Winter Weather Impact (January 2026)")
print("="*80)
print(f"\nSchools with severe weather: {(final_table['severe_weather_days_count'] > 0).sum()}/10")
print(f"Total student-days impacted: {final_table['student_days_impacted'].sum():,.0f}\n")

# Display with formatted numbers
final_table[['school_name', 'city', 'state', 'enrollment', 'severe_weather_days_count', 'student_days_impacted']].style.format({
    'enrollment': '{:,.0f}',
    'student_days_impacted': '{:,.0f}'
})

---
## Step 9: Load to Snowflake

In [ ]:
# load to snowflake for SQL analysis
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.sql("USE DATABASE SNOWBEARAIR_DB").collect()
session.sql("USE SCHEMA PUBLIC").collect()

# Prepare data
snowflake_table = final_table.drop(columns=['enrollment_source'])

# Create table
table_name = 'MUSIC_SCHOOLS_WINTER_WEATHER_IMPACT_JAN2026'

session.sql(f"""
CREATE OR REPLACE TABLE {table_name} (
    SCHOOL_NAME VARCHAR,
    CITY VARCHAR,
    STATE VARCHAR,
    ENROLLMENT NUMBER,
    SEVERE_WEATHER_DAYS_COUNT NUMBER,
    SEVERE_WEATHER_DATES VARCHAR,
    STUDENT_DAYS_IMPACTED NUMBER,
    URL VARCHAR
)
""").collect()

# Insert data
for idx, row in snowflake_table.iterrows():
    session.sql(f"""
    INSERT INTO {table_name} VALUES (
        '{row['school_name'].replace("'", "''")}',
        '{row['city']}',
        '{row['state']}',
        {row['enrollment']},
        {row['severe_weather_days_count']},
        '{row['severe_weather_dates'].replace("'", "''")}',
        {row['student_days_impacted']},
        '{row['url']}'
    )
    """).collect()

print(f"Data loaded to: SNOWBEARAIR_DB.PUBLIC.{table_name}")

---
## Step 10: SQL Analysis

# First, present the Final Output as required by the Assignment Rubric
# Then, conduct assorted other analyses so we know the impact to our trombone and tuba carrying bandmates!

In [ ]:
%%sql

# For each school, where is it, total music school enrollment, number of days of severe weather, and total student days impacted
SELECT 
    SCHOOL_NAME,
    CITY,
    STATE,
    TO_VARCHAR(ENROLLMENT, '999,999') AS ENROLLMENT,
    SEVERE_WEATHER_DAYS_COUNT,
    TO_VARCHAR(STUDENT_DAYS_IMPACTED, '999,999') AS STUDENT_DAYS_IMPACTED
FROM MUSIC_SCHOOLS_WINTER_WEATHER_IMPACT_JAN2026
ORDER BY STUDENT_DAYS_IMPACTED DESC;

In [ ]:
%%sql
#  Same data, but by state - which combines some schools together
WITH state_summary AS (
    SELECT 
        STATE,
        COUNT(*) AS school_count,
        SUM(ENROLLMENT) AS total_enrollment,
        SUM(STUDENT_DAYS_IMPACTED) AS total_impact
    FROM MUSIC_SCHOOLS_WINTER_WEATHER_IMPACT_JAN2026
    GROUP BY STATE
)
SELECT 
    STATE,
    school_count,
    TO_VARCHAR(total_enrollment, '999,999') AS total_enrollment,
    TO_VARCHAR(total_impact, '999,999') AS total_impact,
    ROUND(total_impact / NULLIF(total_enrollment, 0), 1) AS avg_days_per_student
FROM state_summary
ORDER BY total_impact DESC;

In [ ]:
%%sql
#  Percentage of total impacted days, but only for those schools with any impact at all
WITH impact_metrics AS (
    SELECT 
        SCHOOL_NAME,
        ENROLLMENT,
        SEVERE_WEATHER_DAYS_COUNT,
        STUDENT_DAYS_IMPACTED
    FROM MUSIC_SCHOOLS_WINTER_WEATHER_IMPACT_JAN2026
    WHERE SEVERE_WEATHER_DAYS_COUNT > 0
)
SELECT 
    SCHOOL_NAME,
    TO_VARCHAR(ENROLLMENT, '999,999') AS ENROLLMENT,
    SEVERE_WEATHER_DAYS_COUNT,
    TO_VARCHAR(STUDENT_DAYS_IMPACTED, '999,999') AS STUDENT_DAYS_IMPACTED,
    ROUND(100.0 * STUDENT_DAYS_IMPACTED / SUM(STUDENT_DAYS_IMPACTED) OVER (), 1) AS pct_of_total
FROM impact_metrics
ORDER BY STUDENT_DAYS_IMPACTED DESC;

---
## Summary

### Data Collection Results

| Data | Primary Source | Fallback | Actual Result |
|------|---------------|----------|---------------|
## | School Names | Web scraping | Reference CSV | ~8/10 scraped |
## | Enrollment | Web scraping → API | Reference CSV | 7/10 scraped, 2/10 API, 1/10 fallback |
## | Coordinates | Reference CSV | N/A | 10/10 |
## | Weather | Open-Meteo API | N/A | 10/10 |

**Note:** Due to Snowflake network restrictions, scraping and API calls were blocked in the cloud environment. 
The pipeline was run locally in a Jupyter Notebook to demonstrate full functionality. 
The locally-generated results (`music_schools_weather_results.csv`) were then imported into Snowflake for SQL analysis.

### Enrollment Data Sources (from local run)

| School | Source | Enrollment |
|--------|--------|------------|
| Curtis Institute of Music | Scraped | 160 |
| Oberlin Conservatory of Music | Scraped | 540 |
| New England Conservatory of Music | Scraped | 850 |
| Manhattan School of Music | Scraped | 960 |
| Indiana University Jacobs School of Music | Scraped | 1,500 |
| Eastman School of Music | Scraped | 260 |
| San Francisco Conservatory of Music | Scraped | 460 |
| The Juilliard School | API | 460 |
| Berklee College of Music | API | 7,510 |
| USC Thornton School of Music | Fallback | 1,069 |

### Data Engineering Challenges

1. **Network Restrictions:** Snowflake cloud environment inconsistently blocked external website access
2. **API Rate Limits:** College Scorecard API appears to limit requests by some measure or means (I'm not sure)
3. **Inconsistent HTML:** Each site requires custom scraping patterns
4. **Data in PDFs:** Some schools publish enrollment in PDF format only
5. **University vs School Data:** APIs return full university enrollment, not music school specifically (e.g., USC)

### Solution: Hybrid Approach

To overcome Snowflake's network restrictions while demonstrating full pipeline functionality:
1. I ran the complete pipeline locally in a Jupyter Notebook
2. It ran beautifully!
3. I exported results to CSV (`music_schools_weather_results.csv`)
4. Imported CSV into Snowflake for SQL analysis and visualization

### Key Insights

1. A Google search shows enrollment data instantly, but programmatic access requires navigating rate limits, firewalls, and inconsistent data formats. 
2. Now I understand why data engineers say "80% of the work is getting the data."
3. If you are going to play tuba, trombone or cello, you should go to music school in California!

### Next Up

I selected some visualizations to enhance the presentation of this data analysis


In [ ]:
# =============================================================================
# VISUALIZATION 1: Horizontal Bar Chart - Student-Days Impacted
# =============================================================================
# This is the "so what" chart - shows total impact by school

# Sort by impact for visual hierarchy
viz_data = final_table.sort_values('student_days_impacted', ascending=True)

fig, ax = plt.subplots(figsize=(10, 8))

# Create horizontal bars
bars = ax.barh(
    viz_data['school_name'], 
    viz_data['student_days_impacted'],
    color='#2E86AB'
)

# Highlight California schools (zero impact) in different color
for i, (school, impact) in enumerate(zip(viz_data['school_name'], viz_data['student_days_impacted'])):
    if impact == 0:
        bars[i].set_color('#E8E8E8')

# Add value labels on bars
for i, (school, impact) in enumerate(zip(viz_data['school_name'], viz_data['student_days_impacted'])):
    if impact > 0:
        ax.text(impact + 1500, i, f'{impact:,.0f}', va='center', fontsize=9)
    else:
        ax.text(impact + 1500, i, 'No severe weather', va='center', fontsize=9, color='gray')

# Formatting
ax.set_xlabel('Student-Days Impacted', fontsize=12)
ax.set_title('Winter Weather Impact on Top Music Schools\nJanuary 2026', fontsize=14, fontweight='bold')
ax.set_xlim(0, max(viz_data['student_days_impacted']) * 1.15)

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()





In [ ]:
# =============================================================================
# VISUALIZATION 2: Horizontal Bar Chart - Severe Weather Days
# =============================================================================
# This removes enrollment bias - shows raw weather severity

fig, ax = plt.subplots(figsize=(10, 8))

# Sort by severe days
viz_data = final_table.sort_values('severe_weather_days_count', ascending=True)

# Create horizontal bars with gradient based on severity
colors = ['#E8E8E8' if d == 0 else plt.cm.Blues(0.3 + (d / 30) * 0.7) 
          for d in viz_data['severe_weather_days_count']]

bars = ax.barh(
    viz_data['school_name'], 
    viz_data['severe_weather_days_count'],
    color=colors
)

# Add value labels
for i, days in enumerate(viz_data['severe_weather_days_count']):
    if days > 0:
        ax.text(days + 0.3, i, f'{days} days', va='center', fontsize=9)
    else:
        ax.text(days + 0.3, i, 'None', va='center', fontsize=9, color='gray')

# Formatting
ax.set_xlabel('Severe Weather Days in January 2026', fontsize=12)
ax.set_title('Severe Winter Weather Days by Music School Location\nJanuary 2026', fontsize=14, fontweight='bold')
ax.set_xlim(0, 31)

# Add reference line for half the month
ax.axvline(x=15.5, color='red', linestyle='--', alpha=0.5, label='Half of January')
ax.legend(loc='lower right')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()



In [ ]:
# =============================================================================
# VISUALIZATION 3: Scatter Plot - Enrollment vs Severe Days
# =============================================================================
# Shows relationship between school size and weather exposure
# My favorite, by far!

fig, ax = plt.subplots(figsize=(10, 8))

# Filter out zero-impact schools for cleaner viz (or keep them - your choice)
# viz_data = final_table[final_table['severe_weather_days_count'] > 0]
viz_data = final_table.copy()

# Bubble size based on student-days impacted (scaled for visibility)
sizes = (viz_data['student_days_impacted'] / 500) + 50

# Color based on severe weather days
colors = viz_data['severe_weather_days_count']

scatter = ax.scatter(
    viz_data['enrollment'],
    viz_data['severe_weather_days_count'],
    s=sizes,
    c=colors,
    cmap='YlOrRd',
    alpha=0.7,
    edgecolors='black',
    linewidth=0.5
)

# Add school labels
for idx, row in viz_data.iterrows():
    # Shorten long names
    name = row['school_name'].replace('School of Music', '').replace('College of Music', '')
    name = name.replace('Conservatory of Music', 'Conserv.').replace('Institute of Music', 'Inst.')
    name = name.strip()
    
    # Offset labels to avoid overlap
    offset_x = 100
    offset_y = 0.5
    
    ax.annotate(
        name,
        (row['enrollment'], row['severe_weather_days_count']),
        xytext=(offset_x, offset_y),
        textcoords='offset points',
        fontsize=8,
        alpha=0.8
    )

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Severe Weather Days', fontsize=10)

# Formatting
ax.set_xlabel('Total Enrollment', fontsize=12)
ax.set_ylabel('Severe Weather Days', fontsize=12)
ax.set_title('Enrollment vs Weather Severity\nBubble Size = Student-Days Impacted', fontsize=14, fontweight='bold')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add grid for readability
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# STRETCH IDEA 1: Compare January 2026 to January 2025 ("Normal" Year)
# Not much is different
# =============================================================================

def fetch_weather_for_year(latitude, longitude, school_name, year):
    """
    Fetch weather data for a specific January
    """
    url = 'https://archive-api.open-meteo.com/v1/archive'
    
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'start_date': f'{year}-01-01',
        'end_date': f'{year}-01-31',
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'temperature_unit': 'fahrenheit',
        'precipitation_unit': 'inch',
        'timezone': 'America/New_York'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        daily_data = data.get('daily', {})
        
        weather_records = [
            {
                'school_name': school_name,
                'year': year,
                'date': date,
                'temp_max_f': temp_max,
                'temp_min_f': temp_min,
                'precipitation_inches': precip
            }
            for date, temp_max, temp_min, precip in zip(
                daily_data.get('time', []),
                daily_data.get('temperature_2m_max', []),
                daily_data.get('temperature_2m_min', []),
                daily_data.get('precipitation_sum', [])
            )
        ]
        
        return pd.DataFrame(weather_records)
        
    except Exception as e:
        print(f"  Error for {school_name} ({year}): {e}")
        return pd.DataFrame()


# Fetch January 2025 data for comparison
print("Fetching January 2025 weather data for comparison...\n")
weather_2025_list = []

for idx, row in schools_df.iterrows():
    print(f"  {row['name']}...")
    weather_df = fetch_weather_for_year(
        row['latitude'],
        row['longitude'],
        row['name'],
        2025
    )
    if not weather_df.empty:
        weather_2025_list.append(weather_df)
    time.sleep(0.5)

weather_2025_df = pd.concat(weather_2025_list, ignore_index=True)

# Apply same severe weather criteria to 2025
weather_2025_df['extreme_cold'] = weather_2025_df['temp_min_f'] < 20
weather_2025_df['full_freeze'] = weather_2025_df['temp_max_f'] < 32
weather_2025_df['winter_precip'] = (
    (weather_2025_df['precipitation_inches'] > 0.5) & 
    (weather_2025_df['temp_max_f'] < 32)
)
weather_2025_df['is_severe_weather'] = (
    weather_2025_df['extreme_cold'] | 
    weather_2025_df['full_freeze'] | 
    weather_2025_df['winter_precip']
)

# Aggregate 2025 severe days
severe_2025_agg = (
    weather_2025_df[weather_2025_df['is_severe_weather']]
    .groupby('school_name')
    .agg(severe_days_2025=('is_severe_weather', 'count'))
    .reset_index()
)

# Merge with final table to create comparison
comparison_df = final_table[['school_name', 'city', 'state', 'severe_weather_days_count']].copy()
comparison_df = comparison_df.rename(columns={'severe_weather_days_count': 'severe_days_2026'})
comparison_df = comparison_df.merge(severe_2025_agg, on='school_name', how='left')
comparison_df['severe_days_2025'] = comparison_df['severe_days_2025'].fillna(0).astype(int)

# Calculate difference
comparison_df['difference'] = comparison_df['severe_days_2026'] - comparison_df['severe_days_2025']
comparison_df['pct_change'] = (
    (comparison_df['difference'] / comparison_df['severe_days_2025'].replace(0, 1)) * 100
).round(1)

# Add interpretation
comparison_df['assessment'] = comparison_df['difference'].apply(
    lambda x: 'Much Worse' if x >= 5 else ('Worse' if x > 0 else ('Same' if x == 0 else 'Better'))
)

print("\n" + "="*80)
print("YEAR-OVER-YEAR COMPARISON: January 2026 vs January 2025")
print("="*80 + "\n")

comparison_df = comparison_df.sort_values('difference', ascending=False)
print(comparison_df[['school_name', 'state', 'severe_days_2025', 'severe_days_2026', 'difference', 'assessment']].to_string(index=False))

# Summary statistics
avg_2025 = comparison_df['severe_days_2025'].mean()
avg_2026 = comparison_df['severe_days_2026'].mean()
print(f"\nAverage severe days across all schools:")
print(f"  January 2025: {avg_2025:.1f} days")
print(f"  January 2026: {avg_2026:.1f} days")
print(f"  Difference:   {avg_2026 - avg_2025:+.1f} days")


In [ ]:
# =============================================================================
# STRETCH IDEA 2: Severity Index
# =============================================================================
# Creates a weighted score based on HOW extreme the weather was
# Factors:
#   1. Extreme cold intensity (how far below 20°F)
#   2. Consecutive severe days (longer streaks = worse)
#   3. Total precipitation during freezing conditions

print("\n" + "="*80)
print("SEVERITY INDEX CALCULATION")
print("="*80 + "\n")

def calculate_severity_index(school_weather_df):
    """
    Calculate a severity index (0-100) based on:
    - Cold intensity: How far below thresholds
    - Duration: Consecutive severe days
    - Precipitation: Snow/ice accumulation
    """
    if school_weather_df.empty:
        return 0
    
    # Component 1: Cold Intensity Score (0-40 points)
    # Based on how far minimum temps dropped below 20°F
    min_temp = school_weather_df['temp_min_f'].min()
    if min_temp < 20:
        cold_intensity = min(40, (20 - min_temp) * 2)  # 2 points per degree below 20
    else:
        cold_intensity = 0
    
    # Component 2: Duration Score (0-30 points)
    # Based on longest streak of consecutive severe days
    severe_days = school_weather_df['is_severe_weather'].values
    max_streak = 0
    current_streak = 0
    for day in severe_days:
        if day:
            current_streak += 1
            max_streak = max(max_streak, current_streak)
        else:
            current_streak = 0
    duration_score = min(30, max_streak * 3)  # 3 points per consecutive day, max 30
    
    # Component 3: Precipitation Score (0-30 points)
    # Based on total precipitation during freezing conditions
    freezing_precip = school_weather_df[
        school_weather_df['temp_max_f'] < 32
    ]['precipitation_inches'].sum()
    precip_score = min(30, freezing_precip * 10)  # 10 points per inch, max 30
    
    total_score = cold_intensity + duration_score + precip_score
    
    return {
        'cold_intensity': round(cold_intensity, 1),
        'duration_score': round(duration_score, 1),
        'precip_score': round(precip_score, 1),
        'severity_index': round(total_score, 1)
    }


# Calculate severity index for each school
severity_data = []

for school_name in final_table['school_name']:
    school_weather = weather_data_df[weather_data_df['school_name'] == school_name].copy()
    scores = calculate_severity_index(school_weather)
    scores['school_name'] = school_name
    severity_data.append(scores)

severity_df = pd.DataFrame(severity_data)

# Merge with final table
severity_results = final_table[['school_name', 'city', 'state', 'enrollment', 'severe_weather_days_count']].merge(
    severity_df, on='school_name'
)

# Sort by severity index
severity_results = severity_results.sort_values('severity_index', ascending=False)

# Add severity category
def categorize_severity(score):
    if score >= 60:
        return 'Extreme'
    elif score >= 40:
        return 'Severe'
    elif score >= 20:
        return 'Moderate'
    elif score > 0:
        return 'Mild'
    else:
        return 'None'

severity_results['severity_category'] = severity_results['severity_index'].apply(categorize_severity)

print("Severity Index Components:")
print("  - Cold Intensity (0-40): How far temps dropped below 20°F")
print("  - Duration Score (0-30): Longest streak of consecutive severe days")
print("  - Precipitation Score (0-30): Total precipitation during freezing temps")
print("  - Total Severity Index: 0-100 scale\n")

print(severity_results[[
    'school_name', 'state', 'cold_intensity', 'duration_score', 
    'precip_score', 'severity_index', 'severity_category'
]].to_string(index=False))


In [ ]:
# =============================================================================
# VISUALIZATION: Year-over-Year Comparison
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 8))

# Prepare data
comp_viz = comparison_df.sort_values('severe_days_2026', ascending=True)
y_pos = range(len(comp_viz))
bar_height = 0.35

# Create grouped bars
bars_2025 = ax.barh([p - bar_height/2 for p in y_pos], comp_viz['severe_days_2025'], 
                     bar_height, label='January 2025', color='#7FB3D5', alpha=0.8)
bars_2026 = ax.barh([p + bar_height/2 for p in y_pos], comp_viz['severe_days_2026'], 
                     bar_height, label='January 2026', color='#2E86AB', alpha=0.8)

# Add value labels
for i, (d25, d26) in enumerate(zip(comp_viz['severe_days_2025'], comp_viz['severe_days_2026'])):
    ax.text(d25 + 0.3, i - bar_height/2, str(int(d25)), va='center', fontsize=9)
    ax.text(d26 + 0.3, i + bar_height/2, str(int(d26)), va='center', fontsize=9)

ax.set_yticks(y_pos)
ax.set_yticklabels(comp_viz['school_name'])
ax.set_xlabel('Severe Weather Days', fontsize=12)
ax.set_title('January 2026 vs January 2025: Severe Weather Days\nby Music School Location', fontsize=14, fontweight='bold')
ax.legend(loc='lower right')
ax.set_xlim(0, 35)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()



In [ ]:
# =============================================================================
# VISUALIZATION: Severity Index
# =============================================================================

fig, ax = plt.subplots(figsize=(12, 8))

# Sort and prepare data
sev_viz = severity_results.sort_values('severity_index', ascending=True)

# Color by category
category_colors = {
    'Extreme': '#C0392B',
    'Severe': '#E74C3C',
    'Moderate': '#F39C12',
    'Mild': '#F1C40F',
    'None': '#E8E8E8'
}
colors = [category_colors[cat] for cat in sev_viz['severity_category']]

# Create stacked horizontal bars
bars = ax.barh(sev_viz['school_name'], sev_viz['severity_index'], color=colors, edgecolor='black', linewidth=0.5)

# Add value labels
for i, (score, cat) in enumerate(zip(sev_viz['severity_index'], sev_viz['severity_category'])):
    label = f'{score:.0f} ({cat})'
    ax.text(score + 1, i, label, va='center', fontsize=9)

ax.set_xlabel('Severity Index (0-100)', fontsize=12)
ax.set_title('Winter Weather Severity Index by Music School\nJanuary 2026', fontsize=14, fontweight='bold')
ax.set_xlim(0, 100)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, label=cat) for cat, color in category_colors.items() if cat != 'None']
ax.legend(handles=legend_elements, loc='lower right', title='Severity')

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()